In [33]:
from discovery_child_development.getters import get_dataset
import pandas as pd
from nesta_ds_utils.loading_saving import S3
from discovery_child_development import PROJECT_DIR, S3_BUCKET
OUTPUT_PATH = PROJECT_DIR / "outputs/labels/evals_data"
OUTPUT_PATH.mkdir(exist_ok=True, parents=True)
S3_PATH = "data/labels/child_development/evals_data/"

In [34]:
def output_eval_data(df: pd.DataFrame, filename: str) -> None:
    """Saves dataframe to json and uploads to S3"""
    df.to_json(path_or_buf=str(OUTPUT_PATH / filename), orient='records', lines=True)
    S3.upload_file(path_from=str(OUTPUT_PATH / filename), bucket=S3_BUCKET, path_to=S3_PATH + filename)

In [15]:
data = get_dataset("taxonomy_labelling_sample")
data

2024-01-31 12:27:55,227 - root - INFO - File data/labels/taxonomy_classifier/training_validation_data_patents_openalex.jsonl downloaded from discovery-iss to /Users/karlis.kanders/Documents/code/discovery_child_development/inputs/data/labelling/taxonomy/input/training_validation_data_patents_openalex.jsonl


,id,text,source
0,W1519846160,Intricate Identities: Cochlear Implant Users N...,openalex
1,W1523874198,Early Bilingual Education. In the coming years...,openalex
2,W1535586210,The Use of Music Therapy Strategies to Enhance...,openalex
3,W1538577929,Experience-based language acquisition: a compu...,openalex
4,W1556618260,"A new old age?: Exploring the values, attitude...",openalex
...,...,...,...
8833,CN-215456599-U,Environment-friendly children learn drawer for...,patents
8834,US-2021022682-A1,Advanced play environment for screening and ea...,patents
8835,CN-113254836-B,Intelligent child-care knowledge point informa...,patents
8836,CN-110122881-A,A nutritional composition for promoting height...,patents


In [16]:
from discovery_child_development import (
    PROJECT_DIR,
    binary_config,
    labelling_config,
    logging,
)
from discovery_child_development.utils.huggingface_pipeline import (
    load_model,
    load_training_args,
    load_trained_model,
)
from discovery_child_development.getters.binary_classifier.binary_classifier_model import (
    get_binary_classifier_models,
)
from discovery_child_development.utils.huggingface_pipeline import (
    predictions_huggingface,
)

# Paths
S3_MODELS_PATH = "models/binary_classifier/"
PATH_TO = f"{PROJECT_DIR}/outputs/data/models/"
MODEL_FILENAME = (
    f"gpt_labelled_binary_classifier_distilbert_production_True.tar.gz"
)
OUTPUT_FILENAME = labelling_config["OUTPUT_FILENAME"]

if __name__ == "__main__":
    # Save the model locally
    logging.info("Downloading the model...")
    get_binary_classifier_models(
        filename=MODEL_FILENAME, s3_path=S3_MODELS_PATH, path_to=PATH_TO
    )
    model_folder = (
        f"{PATH_TO}gpt_labelled_binary_classifier_distilbert_production_True"
    )

    # Load the model
    logging.info("Loading the model...")
    model = load_model(model_path=model_folder, config=binary_config, num_labels=2)

    # Train model with early stopping
    training_args = load_training_args(**binary_config["training_args"])
    trainer = load_trained_model(
        model=model,
        args=training_args,
        config=binary_config,
    )

    # Get the labelled data
    logging.info("Getting the labelled data...")
    data_for_labelling = data[['id', 'text']]

    # Get the predictions
    logging.info("Getting the predictions...")
    predictions = predictions_huggingface(
        trainer=trainer, text_data=data_for_labelling, config=binary_config
    )

    # # Save the predictions
    # logging.info("Saving the predictions...")
    # S3.upload_obj(
    #     predictions,
    #     S3_BUCKET,
    #     f"data/outputs/binary_classifier/{OUTPUT_FILENAME}.csv",
    # )


2024-01-31 12:33:35,087 - root - INFO - Downloading the model...
2024-01-31 12:33:56,953 - root - INFO - Loading the model...
2024-01-31 12:33:57,608 - root - INFO - Getting the labelled data...
2024-01-31 12:33:57,617 - root - INFO - Getting the predictions...


100%|██████████| 553/553 [03:31<00:00,  2.61it/s]


In [19]:
from discovery_child_development.getters.labels import get_taxonomy_labels

In [20]:
tax_labels = get_taxonomy_labels()

In [35]:
labelled_data = (
    predictions
    .query("predictions == 1")
    .merge(tax_labels[['id', 'source', 'prediction']], left_on="id", right_on="id", how="left")
    .drop(columns=["predictions"])
    .explode("prediction")
)

In [37]:
labelled_data.sample()

,id,text,source,prediction
822,W3118471363,SOCIAL LEARNING IN THE CHILDHOOD: INFLUENCE OF...,openalex,Family and home


In [31]:
labelled_data.groupby("prediction").count()

,id,text
prediction,,
AR VR,37,37
Assessment (general),1085,1085
Child protection,620,620
Cognitive development,1239,1239
Communication and language,478,478
Community,741,741
Data,615,615
Data science and AI,209,209
Expressive arts and design,402,402


In [39]:
import numpy as np

def sample_n_predictions_stratified(
    df: pd.DataFrame,
    n: int,
    column_A: str = 'prediction',
    column_B: str= 'source',
    seed: int = 42
) -> pd.DataFrame:
    """
    Produce a sample with n elements for each label in column_A, uniformly distributed by column_B labels

    Args:
        df (pd.DataFrame): dataframe with predictions
        n (int): number of samples
        column_A (str, optional): column to groupby. Defaults to 'prediction'.
        column_B (str, optional): column to stratify. Defaults to 'source'.
        seed (int, optional): random seed. Defaults to 42.

    Returns:
        pd.DataFrame: dataframe with n samples for each unique value of column prediction
    """
    np.random.seed(seed)
    return (
        df
        .groupby([column_A, column_B])
        .apply(lambda x: x.sample(n=n//2))
        .reset_index(drop=True)
    )

In [44]:
unique_labels = labelled_data['prediction'].unique()

# write a function that goes through each label
# and samples 25 examples from each source - OR
# if there are not enough samples, sample all of them
# and then sample the remaining from the other sources
def  sample_n_or_all(
    df: pd.DataFrame,
    n: int,
    column_A: str = 'prediction',
    column_B: str= 'source',
    seed: int = 42
):
    """
    """
    np.random.seed(seed)
    unique_Bs = df[column_B].unique()
    sampled = pd.DataFrame()
    for label in unique_labels:
        label_df = df.query(f"{column_A} == '{label}'")
        for source in unique_Bs:
            if len(label_df.query(f"{column_B} == '{source}'")) < n:
                sampled = pd.concat([sampled, label_df.query(f"{column_B} == '{source}'")])
            else:
                sampled = pd.concat([sampled, label_df.query(f"{column_B} == '{source}'").sample(n=n)])
    return sampled

def sample_data(df, column_A, column_B, n, seed):
    np.random.seed(seed)
    return df.groupby([column_A, column_B]).apply(lambda x: x.sample(min(len(x), n))).reset_index(drop=True)

In [80]:
sample = sample_n_or_all(labelled_data, 25, 'prediction', 'source', seed=100) 
sample2 = sample_data(labelled_data, 'prediction', 'source', 25, seed=100)

In [81]:
# replace the label "Family and home" to "Parenting" in prediction column
# sample['prediction'] = sample['prediction'].replace('Family and home', 'Parenting')

In [82]:
sample.groupby("prediction").count().sort_values(by="prediction")

,id,text,source
prediction,,,
AR VR,36,36,36
Assessment (general),50,50,50
Child protection,50,50,50
Cognitive development,50,50,50
Communication and language,50,50,50
Community,39,39,39
Data,50,50,50
Data science and AI,50,50,50
Expressive arts and design,50,50,50


In [85]:
pd.set_option('display.max_colwidth', None)
sample = sample.sort_values("prediction")

In [ ]:
# add "https://openalex.org/works/" to the openalex IDs

In [86]:
sample

,id,text,source,prediction
1686,W4285741356,"Mobile Augmented Reality applied as a learning strategy for early childhood education students.. The purpose of this research is to determine the effect of the use of mobile augmented reality applied as a learning strategy for early childhood education students. The research is applied, pre-experimental and quantitative design. The population consisted of children in the initial grade of an educational institution in Peru. The sample consisted of 20 students of 5 years old as experimental and control group. As a result of the intervention, it was possible to improve learning through competencies in 80% of an expected level of achievement and 20% of an outstanding level of achievement, concluding that the use of mobile augmented reality improved learning in early education. The use of this technology is recommended as a pedagogical tool for the early education sector in Peru.",openalex,AR VR
1524,W4241367487,"IPads in The Early Childhood Science Classroom: An Exploration of IPads Use in Hands - On Science Activities. This qualitative case study explored the affordances and uses of the iPad in Prep classrooms when integrated into hands-on science activities about movement. Using a hybrid inductive and deductive thematic analysis approach, this study identified and described the ways that virtual and augmented reality apps on the iPad can both enhance and distract from hands-on science activities about movement in Prep. The findings suggest ways that educators may achieve greater integration of hands-on science and technology learning experiences in Prep classrooms.",openalex,AR VR
3874,KR-102520287-B1,"Kids care platform service system using metaverse and method for controlling the same. A child care service system and control method using the metaverse are disclosed. The present invention synchronizes information of daycare centers or kindergarten children and provides them in real time, thereby assisting with rapid confirmation of children&#39;s safety and developmental status, safety status of facilities, etc., and childcare tasks in the real world.",patents,AR VR
1737,W4293868984,"Research on the Design and Effect of Early Childhood Education Games Supported by AR Technology. The main content of this study is the research on the design and effect of early childhood educational games supported by augmented reality (hereinafter referred to as AR) technology. The research starts with the investigation and literature review of practical problems in early childhood education, aiming at the practical problems, combined with the characteristics of AR technology, and systematically proposes a preschool education communication process model supported by technology to solve the practical problems of early childhood education. Using the characteristics of technological multimedia integration, establish an all-media integrated resource package for early childhood education to solve practical problems such as scattered educational resources, duplication of construction, and resource waste technology.",openalex,AR VR
2517,W4320159718,"Establishment of Childhood's Discipline Characters in a Clean and Healthy Life Behavior in The Pandemic Covid-19. Clean and healthy living behavior is a big challenge in early childhood education, especially during the Clean and healthy living behavior is a big challenge in early childhood education, especially during the covid 19 pandemic. One of the primary keys, in this case, is the disciplined character of children. This study aims to analyze the character of children's discipline in implementing Clean and Healthy Life Behavior during the Covid-19 Pandemic. This research is a type of qualitative research with a descriptive approach. Data analysis was carried out by systematically analyzing data obtained from interviews, field notes, and other materials so that they were quickly accessible and informed to others. The results showed that children's discipline in

In [87]:
filename = "taxonomy_labels_eval.jsonl"
output_eval_data(sample, filename)

In [88]:
sample.groupby("source").count()

,id,text,prediction
source,,,
openalex,1050,1050,1050
patents,759,759,759


In [90]:
# Import categories and export as a table
from discovery_child_development.utils import taxonomy_labelling_utils as tlu
PATH_TO_CATEGORIES = PROJECT_DIR / "discovery_child_development/pipeline/labelling/taxonomy/prompts/categories.json"
categories = tlu.load_categories(PATH_TO_CATEGORIES)

In [98]:
# turn the dict into a table
categories_table = (
    pd.DataFrame.from_dict(categories, orient="index").reset_index().rename(columns={"index": "category", 0: "description"})
    .to_csv(str(PROJECT_DIR / "outputs/labels/taxonomy/categories.csv"), index=False)
)
